# Economie sociale et solidaire

D'après le fichier INSEE des entreprises:
- Il y a plus de 1.1 million d'entreprises répertoriées.
- 95.8% sont des associations (déclarées, droit local, ...). 
- 98.35% des entreprises ESS sont encore actives. (etatAdministratifUniteLegale)
- Seulement 1.64% ont cessé leur activités. 
- 18.11% des entreprises ESS embauchent des employés (caractereEmployeurUniteLegale)
- Il y a 153 ESS qui font de la programmation informatique. Parmi celles-ci, 84 ont un statut juridique de coopérative
- Par exemple: [Une coopérative de maintenance informatique (pour des agriculteurs?)](https://www.societe.com/societe/services-et-maintenance-informatique-313167116.html)

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append("/".join(os.getcwd().split("/")[:-2]) + "/")

In [9]:
from katalogue.insee.catégories_juridiques import DépôtINSEECatégoriesJuridiques
from katalogue.insee.arborescence_naf import DépôtINSEEArborescenceNAF
from katalogue.insee.sirene_entreprises import DépôtINSEESireneEntreprises
import pandas as pd

dépôt = DépôtINSEECatégoriesJuridiques()
cat_juridiques = dépôt.catégories(niveau=3)

dépôt = DépôtINSEEArborescenceNAF()
NAF = dépôt.naf()



In [16]:
ess_fn = os.path.expanduser("~/.katalogue/ess.pkl")
if not(os.path.exists(ess_fn)):
    dépôt = DépôtINSEESireneEntreprises()
    ess = dépôt.entreprises_économie_sociale_et_solidaire(nombre_maximum_entreprises=None)
    ess.to_pickle(ess_fn)
else:
    ess = pd.read_pickle(ess_fn)

In [53]:
ess.etatAdministratifUniteLegale.value_counts()

A    1137702
C      19062
Name: etatAdministratifUniteLegale, dtype: int64

In [63]:
ess.caractereEmployeurUniteLegale.value_counts()

N    946306
O    209586
Name: caractereEmployeurUniteLegale, dtype: int64

In [ ]:
assocs = ess.merge(cat_juridiques[cat_juridiques.libellé.str.contains("Association")],
          left_on="categorieJuridiqueUniteLegale", 
          right_on="code")

In [67]:
assocs.libellé.value_counts()

Association déclarée                                                   1080387
Association de droit local (Bas-Rhin, Haut-Rhin et Moselle)              27869
Association déclarée, reconnue d'utilité publique                         2114
Association déclarée d'insertion par l'économique                          653
Association intermédiaire                                                  579
Association coopérative inscrite (droit local Alsace Moselle)              316
Association non déclarée                                                    23
Association syndicale libre                                                  8
Association d'avocats à responsabilité professionnelle individuelle          4
Name: libellé, dtype: int64

In [74]:
assocs.merge(NAF, left_on="activitePrincipaleUniteLegale", right_on="id_5").label_5.value_counts().head(10)

Autres organisations fonctionnant par adhésion volontaire    552497
Activités de clubs de sports                                 169147
Arts du spectacle vivant                                      97382
Action sociale sans hébergement n.c.a.                        36741
Autres activités récréatives et de loisirs                    27603
Autres activités liées au sport                               20702
Activités de soutien au spectacle vivant                      14786
Autres enseignements                                          13720
Formation continue d'adultes                                  13104
Enseignement culturel                                         12750
Name: label_5, dtype: int64

In [87]:
entreprises = ess[~ess.categorieJuridiqueUniteLegale.isin(cat_juridiques[cat_juridiques.libellé.str.contains("Association")].code.values)]\
    .merge(NAF, left_on="activitePrincipaleUniteLegale", right_on="id_5")\
    .merge(cat_juridiques,
          left_on="categorieJuridiqueUniteLegale", 
          right_on="code")

In [105]:
entreprises.label_5.value_counts().head(50)

Autre mise à disposition de ressources humaines                                                                                 8352
Activités de soutien aux cultures                                                                                               7222
Location et location-bail de machines et équipements agricoles                                                                  6571
Autres intermédiations monétaires                                                                                               3596
Autres assurances                                                                                                               2748
Autres organisations fonctionnant par adhésion volontaire                                                                       1584
Supports juridiques de programmes                                                                                                849
Vinification                                                         

In [102]:
prog_informatique = entreprises[entreprises.label_5.eq("Programmation informatique")]

prog_informatique.shape[0]

153

In [103]:
prog_informatique.libellé.value_counts()

SAS, société par actions simplifiée                                  65
Autre SARL coopérative                                               52
SARL coopérative de production (SCOP)                                20
Société à responsabilité limitée (sans autre indication)              5
Autre SA coopérative à conseil d'administration                       4
Union de sociétés coopératives agricoles                              3
Autre société civile coopérative                                      1
SA coopérative de production  (SCOP) à conseil d'administration       1
SA d'intérêt collectif agricole (SICA) à conseil d'administration     1
SARL coopérative artisanale                                           1
Name: libellé, dtype: int64

In [100]:
prog_informatique[["siren", "dateCreationUniteLegale", "libellé", "trancheEffectifsUniteLegale"]]

,siren,dateCreationUniteLegale,libellé,trancheEffectifsUniteLegale
507,317495646,1979-01-01,Autre SA coopérative à conseil d'administration,21
508,439192287,2001-07-23,Autre SA coopérative à conseil d'administration,NN
509,453961690,2004-04-13,Autre SA coopérative à conseil d'administration,NN
510,818054207,2016-02-15,Autre SA coopérative à conseil d'administration,02
1622,795009919,2013-08-20,"SAS, société par actions simplifiée",11
...,...,...,...,...
13779,313167116,1978-01-01,SA d'intérêt collectif agricole (SICA) à conse...,NN
17076,313967945,1978-01-01,Union de sociétés coopératives agricoles,12
17077,314175613,1977-07-01,Union de sociétés coopératives agricoles,NN
17078,341452522,1987-06-11,Union de sociétés coopératives agricoles,NN


In [106]:
restauration_rapide = entreprises[entreprises.label_5.eq("Restauration de type rapide")]

In [108]:
restauration_rapide.libellé.value_counts()

SAS, société par actions simplifiée                         31
Société à responsabilité limitée (sans autre indication)    17
SARL coopérative de production (SCOP)                       13
Autre SARL coopérative                                      12
SARL coopérative artisanale                                  1
SARL union de sociétés coopératives                          1
Name: libellé, dtype: int64

In [109]:
restauration_rapide

,siren,statutDiffusionUniteLegale,unitePurgeeUniteLegale,dateCreationUniteLegale,sigleUniteLegale,sexeUniteLegale,prenom1UniteLegale,prenom2UniteLegale,prenom3UniteLegale,prenom4UniteLegale,...,id_2,label_2,id_3,label_3,id_4,label_4,id_5,label_5,code,libellé
2275,828292490,O,NaN,2017-03-13,NaN,NaN,NaN,NaN,NaN,NaN,...,56,Restauration,56.1,Restaurants et services de restauration mobile,56.1,Restaurants et services de restauration mobile,56.10C,Restauration de type rapide,5710,"SAS, société par actions simplifiée"
2276,832108260,O,NaN,2017-09-30,MG,NaN,NaN,NaN,NaN,NaN,...,56,Restauration,56.1,Restaurants et services de restauration mobile,56.1,Restaurants et services de restauration mobile,56.10C,Restauration de type rapide,5710,"SAS, société par actions simplifiée"
2277,838366268,O,NaN,2018-03-20,NaN,NaN,NaN,NaN,NaN,NaN,...,56,Restauration,56.1,Restaurants et services de restauration mobile,56.1,Restaurants et services de restauration mobile,56.10C,Restauration de type rapide,5710,"SAS, société par actions simplifiée"
2278,842539652,O,NaN,2018-09-19,NaN,NaN,NaN,NaN,NaN,NaN,...,56,Restauration,56.1,Restaurants et services de restauration mobile,56.1,Restaurants et services de restauration mobile,56.10C,Restauration de type rapide,5710,"SAS, société par actions simplifiée"
2279,842665606,O,NaN,2018-05-25,NaN,NaN,NaN,NaN,NaN,NaN,...,56,Restauration,56.1,Restaurants et services de restauration mobile,56.1,Restaurants et services de restauration mobile,56.10C,Restauration de type rapide,5710,"SAS, société par actions simplifiée"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12189,902219948,O,NaN,2021-08-09,NaN,NaN,NaN,NaN,NaN,NaN,...,56,Restauration,56.1,Restaurants et services de restauration mobile,56.1,Restaurants et services de restauration mobile,56.10C,Restauration de type rapide,5499,Société à responsabilité limitée (sans autre i...
12190,904988243,O,NaN,2021-11-04,NaN,NaN,NaN,NaN,NaN,NaN,...,56,Restauration,56.1,Restaurants et services de restauration mobile,56.1,Restaurants et services de restauration mobile,56.10C,Restauration de type rapide,5499,Société à responsabilité limitée (sans autre i...
12191,910404433,O,NaN,2022-01-14,M C,NaN,NaN,NaN,NaN,NaN,...,56,Restauration,56.1,Restaurants et services de restauration mobile,56.1,Restaurants et services de restauration mobile,56.10C,Restauration de type rapide,5499,Société à responsabilité limitée (sans autre i...
31382,820413664,O,NaN,2016-05-01,NaN,NaN,NaN,NaN,NaN,NaN,...,56,Restauration,56.1,Restaurants et services de restauration mobile,56.1,Restaurants et services de restauration mobile,56.10C,Restauration de type rapide,5453,SARL coopérative artisanale
